In [20]:
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler
import folium
import networkx as nx
from haversine import haversine

In [21]:
airport_cols = [
    "Airport ID", "Name", "City", "Country", "IATA", "ICAO",
    "Latitude", "Longitude", "Altitude", "Timezone", "DST",
    "Tz database time zone", "Type", "Source"
]

airports_df = pd.read_csv("airports.dat", header=None, names=airport_cols)
airports_df = airports_df.dropna(subset=["IATA"])
airports_df = airports_df[airports_df["IATA"] != "\\N"]

airport_to_idx = {code: i for i, code in enumerate(airports_df["IATA"])}
idx_to_airport = {v: k for k, v in airport_to_idx.items()}

In [22]:
coords = {
    code: (row["Latitude"], row["Longitude"])
    for code, row in airports_df.set_index("IATA")[["Latitude", "Longitude"]].iterrows()
}

In [23]:
route_cols = [
    "Airline", "Airline ID", "Source Airport", "Source Airport ID",
    "Destination Airport", "Destination Airport ID", "Codeshare",
    "Stops", "Equipment"
]

routes_df = pd.read_csv("routes.dat", header=None, names=route_cols)
routes_df = routes_df.dropna(subset=["Source Airport", "Destination Airport"])

In [24]:
edges = []
for _, row in routes_df.iterrows():
    src, dst = row["Source Airport"], row["Destination Airport"]
    if src in airport_to_idx and dst in airport_to_idx:
        coord_src = coords[src]
        coord_dst = coords[dst]
        dist = haversine(coord_src, coord_dst)
        edges.append((airport_to_idx[src], airport_to_idx[dst], dist))

In [25]:
edge_index = torch.tensor([[e[0] for e in edges], [e[1] for e in edges]], dtype=torch.long)
edge_attr = torch.tensor([e[2] for e in edges], dtype=torch.float).unsqueeze(1)

features = airports_df[["Latitude", "Longitude", "Altitude"]].values
scaler = StandardScaler()
x = torch.tensor(scaler.fit_transform(features), dtype=torch.float)

data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

In [29]:
class GCN_Autoencoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder1 = GCNConv(3, 16)
        self.encoder2 = GCNConv(16, 8)
        self.decoder = torch.nn.Linear(8, 3)  # Output matches input

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.encoder1(x, edge_index))
        x = F.relu(self.encoder2(x, edge_index))
        return self.decoder(x)

model = GCN_Autoencoder()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [30]:
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.mse_loss(out, data.x)  # Now both are [N, 3]
    loss.backward()
    optimizer.step()
print("✅ GCN Autoencoder training done.")

✅ GCN Autoencoder training done.


In [31]:
G = nx.DiGraph()
for src, dst, dist in edges:
    G.add_edge(src, dst, weight=dist)

In [41]:
src_code = "LAX"  # Raipur
dst_code = "BOM"  # Los Angeles

In [42]:
src_idx = airport_to_idx.get(src_code)
dst_idx = airport_to_idx.get(dst_code)

In [43]:
if src_idx is None or dst_idx is None:
    print("❌ One or both airports not found in the graph.")
elif not nx.has_path(G, src_idx, dst_idx):
    print(f"❌ No path between {src_code} and {dst_code}.")
else:
    shortest_path = nx.dijkstra_path(G, source=src_idx, target=dst_idx, weight="weight")
    print("✈️ Shortest Path (IATA):", [idx_to_airport[i] for i in shortest_path])

✈️ Shortest Path (IATA): ['LAX', 'PEK', 'LXA', 'KTM', 'BOM']


In [44]:
path_coords = [coords[idx_to_airport[i]] for i in shortest_path]
m = folium.Map(location=path_coords[0], zoom_start=4)

In [45]:
for i in range(len(path_coords) - 1):
    folium.Marker(path_coords[i], tooltip=idx_to_airport[shortest_path[i]]).add_to(m)
    folium.PolyLine([path_coords[i], path_coords[i + 1]], color="blue", weight=3).add_to(m)

folium.Marker(path_coords[-1], tooltip=idx_to_airport[shortest_path[-1]], icon=folium.Icon(color='red')).add_to(m)
m